In [ ]:
using DataFrames, MixedModels, StatsBase, Gadfly, CSV, CategoricalArrays, Colors

In [ ]:
function gen_brew_colors(n)
    cs = distinguishable_colors(n, 
        [colorant"#66c2a5", colorant"#fc8d62", colorant"#8da0cb", colorant"#e78ac3", 
            colorant"#a6d854", colorant"#ffd92f", colorant"#e5c494", colorant"#b3b3b3"],
        lchoices=Float64[58, 45, 72.5, 90],
        transform=c -> deuteranopic(c, 0.1),
        cchoices=Float64[20,40],
        hchoices=[75,51,35,120,180,210,270,310]
    )
    convert(Vector{Color}, cs)
end

set_default_plot_size(9inch, 9inch/MathConstants.golden)

Import the data from Douven and Schupbach (2015a):

In [ ]:
data = CSV.read("data.csv");

Add difference in explanatory goodness as well as identifiers for participants:

In [ ]:
data[!, :diff] = Array{Float64}(data[!, :Judge_A] .- data[!, :Judge_B])
data[!, :id] = CategoricalArray(repeat(1:26, inner=10))
first(data, 6)

Below the model that came out on top in Douven and Schupbach's analysis, MMOD:

In [ ]:
form = @formula(Post_AS ~ Post_AO + diff + (Post_AO + diff | id));

In [ ]:
mm = LinearMixedModel(form, data);

In [ ]:
fit!(mm)

Below the Bayesian model, MMO:

In [ ]:
form₀ = @formula(Post_AS ~ Post_AO + (Post_AO | id));

In [ ]:
mm₀ = LinearMixedModel(form₀, data);

In [ ]:
fit!(mm₀)

Finally, the model MMOAB:

In [ ]:
form₁ = @formula(Post_AS ~ Post_AO + Judge_A + Judge_B + (Post_AO + Judge_A + Judge_B | id));

In [ ]:
mm₁ = LinearMixedModel(form₁, data);

In [ ]:
fit!(mm₁)

In [ ]:
data1 = copy(data)

ProbEvA = Array{Float64}(undef, 10, 26)
ProbEvB = Array{Float64}(undef, 10, 26)

for i in 1:26
    
    Pr = vcat(0.5, data1[!, :Post_AO][(i * 10 - 9):(i * 10)])

    for j in 1:10
        Pr[j + 1] < Pr[j] ? (ProbEvA[j, i], ProbEvB[j, i]) = (0.25, 0.625) : (ProbEvA[j, i], ProbEvB[j, i]) = (0.75, 0.375)
    end

end

data1[!, :ProbEv] = mean(hcat(cumprod(ProbEvA, dims=1)[:], cumprod(ProbEvB, dims=1)[:]), dims=2)[:]
data1[!, :HA_Ev] = data1[!, :Post_AO] .* data1[!, :ProbEv];

In [ ]:
first(data1, 6)

In [ ]:
function aic_vals(d)
    data1[!, :Prob_noise] = ((1-2d)^2 .* data1[!, :HA_Ev] .+ 
        (d-2d^2) .* (0.5 .+ data1[!, :ProbEv]) .+ d^2) ./ ((1-2d) .* data1[!, :ProbEv] .+ d)
    BAY = aic(fit!(LinearMixedModel(@formula(Post_AS ~ Prob_noise + (1 + Prob_noise | id)), data1)))
    EXPL = aic(fit!(LinearMixedModel(@formula(Post_AS ~ Prob_noise + Judge_A + Judge_B + 
                            (1 + Prob_noise + Judge_A + Judge_B | id)), data1)))
    DIFF = aic(fit!(LinearMixedModel(@formula(Post_AS ~ Prob_noise + diff + (1 + Prob_noise + diff | id)), data1)))
    return vcat(BAY, EXPL, DIFF)
end

In [ ]:
aicVals = Array{Float64, 2}(undef, 100, 3)

for i in 1:100
    aicVals[i,:] = aic_vals(collect(0.0:0.005:0.495)[i])
end

In [ ]:
AV = DataFrame(AIC = aicVals[:],
               d_val = repeat(collect(0.000:0.005:0.495), outer=3),
               Predictors = repeat(["<i>f</i>(O, <i>d</i>)", "<i>f</i>(O, <i>d</i>)AB", "<i>f</i>(O, <i>d</i>)D"], inner=100));

Same for BIC values:

In [ ]:
function bic_vals(d)
    data1[!, :Prob_noise] = ((1-2d)^2 .* data1[!, :HA_Ev] .+ 
        (d-2d^2) .* (0.5 .+ data1[!, :ProbEv]) .+ d^2) ./ ((1-2d) .* data1[!, :ProbEv] .+ d)
    BAY = bic(fit!(LinearMixedModel(@formula(Post_AS ~ Prob_noise + (1 + Prob_noise | id)), data1)))
    EXPL = bic(fit!(LinearMixedModel(@formula(Post_AS ~ Prob_noise + Judge_A + Judge_B + 
                            (1 + Prob_noise + Judge_A + Judge_B | id)), data1)))
    DIFF = bic(fit!(LinearMixedModel(@formula(Post_AS ~ Prob_noise + diff + (1 + Prob_noise + diff | id)), data1)))
    return vcat(BAY, EXPL, DIFF)
end

In [ ]:
bicVals = Array{Float64, 2}(undef, 100, 3)

for i in 1:100
    bicVals[i,:] = bic_vals(collect(0.0:0.005:0.495)[i])
end

In [ ]:
BV = DataFrame(BIC = bicVals[:],
               d_val = repeat(collect(0.000:0.005:0.495), outer=3),
               Predictors = repeat(["<i>f</i>(O, <i>d</i>)", "<i>f</i>(O, <i>d</i>)AB", "<i>f</i>(O, <i>d</i>)D"], inner=100));

Visualize the outcomes:

In [ ]:
CWaic = plot(AV, x=:d_val, y=:AIC, color=:Predictors, Geom.point,
    Guide.ylabel("AIC value"),
    Guide.xlabel("<i>d</i>"),
    Scale.color_discrete_manual(gen_brew_colors(3)...),
    Theme(point_size=2.5pt, minor_label_font_size=10pt,
        major_label_font_size=14pt,
        minor_label_color=colorant"black",
        major_label_color=colorant"black",
        key_title_font_size=13pt,
        key_label_font_size=11pt,
        key_label_color=colorant"black",
        key_title_color=colorant"black",
        highlight_width=0.25pt))

In [ ]:
CWbic = plot(BV, x=:d_val, y=:BIC, color=:Predictors, Geom.point,
    Guide.ylabel("BIC value"),
    Guide.xlabel("<i>d</i>"),
    Scale.color_discrete_manual(gen_brew_colors(3)...),
    Theme(point_size=2.5pt, minor_label_font_size=10pt,
        major_label_font_size=14pt,
        minor_label_color=colorant"black",
        major_label_color=colorant"black",
        key_title_font_size=13pt,
        key_label_font_size=11pt,
        key_label_color=colorant"black",
        key_title_color=colorant"black",
        highlight_width=0.25pt))